In [1]:
import ee
import geemap

In [2]:
#ee.Authenticate()
ee.Initialize(project='ee-dabrowaremote')

In [3]:
aoi = ee.Geometry.Polygon(
    [[[18.8200, 50.4000], [18.9600, 50.4000], [18.9600, 50.3000], [18.8200, 50.3000]]]
)
start_date = '2023-06-01'
end_date = '2023-09-01'


In [ ]:
"""
def mask_clouds(image):
        # Use the Scene Classification Layer (SCL) to mask clouds and shadows
        scl = image.select('SCL')
        cloud_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
        return image.updateMask(cloud_mask)
"""

In [5]:
def mask_qa60(image):
    qa = image.select('QA60')
    
    # Bit 10 = chmura, Bit 11 = cień chmury
    cloud_bit_mask = 1 << 10
    shadow_bit_mask = 1 << 11

    # Bitwise AND to sprawdzenie, czy bity są ustawione
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(shadow_bit_mask).eq(0))

    return image.updateMask(mask)

In [6]:
# Load Sentinel-2 surface reflectance data
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(mask_qa60) \
    .map(lambda img: img.clip(aoi))

# Create mosaic using the best (least cloudy) pixels
mosaic = s2.median()


c:\Users\tomas\miniconda3\envs\RemoteSensing\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
#Funkcja do pakietu
def create_cloud_free_mosaic(aoi, start_date, end_date):
    # Load Sentinel-2 surface reflectance data
    def mask_qa60(image):
        qa = image.select('QA60')
        
        # Bit 10 = chmura, Bit 11 = cień chmury
        cloud_bit_mask = 1 << 10
        shadow_bit_mask = 1 << 11

        # Bitwise AND to sprawdzenie, czy bity są ustawione
        mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
            qa.bitwiseAnd(shadow_bit_mask).eq(0))

        return image.updateMask(mask)

    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(mask_qa60) \
        .map(lambda img: img.clip(aoi))

    # Create mosaic using the best (least cloudy) pixels
    mosaic = s2.median()

    return mosaic

In [7]:
true_color_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

centroid = aoi.centroid()
m = geemap.Map(center=[centroid.getInfo()['coordinates'][1], centroid.getInfo()['coordinates'][0]], zoom=10)
m.addLayer(mosaic, true_color_params, "Mosaic")
m

Map(center=[50.35000345854823, 18.89000000000224], controls=(WidgetControl(options=['position', 'transparent_b…